In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, count, sum, avg, max, min, stddev, mean, udf, round
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.window import Window
import sys

In [ ]:
from pyspark.sql.functions import year, month, dayofweek

# Initialize Spark session
spark = SparkSession.builder.appName("StagingTransform").getOrCreate()

# Read the Parquet file into a DataFrame
spark_payment_df = spark.read.parquet('s3://bucket-name/path/to/output/payment/') # or redshift

# same for product_df
spark_product_df = spark.read.parquet('s3://bucket-name/path/to/output/payment/') # or redshift

In [ ]:
# Feature Engineering

# Extract time-based features. for each row, extract the year, month, and day of the week
spark_payment_df = spark_payment_df.withColumn("transaction_year", year("transaction_date"))
spark_payment_df = spark_payment_df.withColumn("transaction_month", month("transaction_date"))
spark_payment_df = spark_payment_df.withColumn("transaction_day_of_week", dayofweek("transaction_date"))

# Calculate days since last transaction (for each customer), so we can see if the customer is active or not
window_spec = Window.partitionBy("customer_id").orderBy("transaction_date") # Window specification for the window function
spark_payment_df = spark_payment_df.withColumn("days_since_last_transaction", 
                                               datediff("transaction_date", lag("transaction_date").over(window_spec))) # lag is the previous row by partition via window specification

# Create a binary feature for high-value transactions, so we can see if the transaction is a high value transaction
avg_payment = spark_payment_df.select(avg("payment_amount")).first()[0]
spark_payment_df = spark_payment_df.withColumn("is_high_value_transaction", 
                                               when(col("payment_amount") > avg_payment, 1).otherwise(0))

# Calculate customer lifetime value (CLV), this is the total amount of money a customer will spend on the business, it implies customer loyalty
clv_df = spark_payment_df.groupBy("customer_id").agg(sum("payment_amount").alias("customer_lifetime_value"))
spark_payment_df = spark_payment_df.join(clv_df, on="customer_id", how="left")

# Create a binary feature for preferred payment method, this is to see if the customer is using their preferred payment method
# we extract it by grouping the payment method by customer and ordering it by the count of the payment method, then we take the first row, which gives us the most used payment method
preferred_payment_method = spark_payment_df.groupBy("customer_id", "payment_method").count() \
                                           .withColumn("rank", row_number().over(Window.partitionBy("customer_id").orderBy(desc("count")))) \
                                           .filter(col("rank") == 1) \
                                           .select("customer_id", "payment_method").withColumnRenamed("payment_method", "preferred_payment_method")
spark_payment_df = spark_payment_df.join(preferred_payment_method, on="customer_id", how="left") \
                                   .withColumn("is_preferred_payment_method", 
                                               when(col("payment_method") == col("preferred_payment_method"), 1).otherwise(0))


In [ ]:
#strip time from transaction_date
spark_payment_df = spark_payment_df.withColumn("transaction_date", col("transaction_date").cast(DateType()))